In [1]:
### Name: Nicholas Miyamoto-Pennywell
import re


# Assignment 2: Analyzing large datasets with Spark.


For this assignment, you will need to make sure you're running from a PySpark docker environment I introduced in class. You can start the docker pySpark docker environment using the following command:

```
docker run --rm -p 4040:4040 -p 8888:8888 -v $(pwd):/home/jovyan/work jupyter/all-spark-notebook
```

Make sure you run the command from the directory containing this jupyter notebook and your data folder.


</b>
# WARNING: For some reason, ipynbb document didn't always sync properly when I was pushing to github. As such, please push often and make sure your incremental changes appear on GitHub.
</b>

### Part 1

The first part will use Spark to analyze the following books, which I have downloaded for you to use from Project Gutenberg. The files are saved to the data folder.

| File name | Book Title|
|:---------:|:----------|
|43.txt | The Strange Case of Dr. Jekyll and Mr. Hyde by Robert Louis Stevenson|
|84.txt | Frankenstein; Or, The Modern Prometheus by Mary Wollstonecraft Shelley |
|398.txt  | The First Book of Adam and Eve by Rutherford Hayes Platt|
|3296.txt | The Confessions of St. Augustine by Bishop of Hippo Saint Augustine|

The objective is to explore whether we can detect similarity between books within the same topic using word-based similarity. 

The task of identifying similar texts in Natural Language Processing is crucial. A naive method for determining whether two documents are similar is to treat them as collections of words (bag of words) and use the number of words they share as a proxy for their similarity. It makes sense that two books with religion as the topic (e.g.  `398.txt` and `3296.txt`) would have more words in common than a book that discusses religion and a book that discusses science fiction (e.g. books `84.txt` and `398.txt`). 

As mentioned above, we will be using Spark to analyze the data. Although Spark is not needed for such a small example, the platform would be ideal for analyzing very large collections of documents, like those often analyzed by large corporations

This part of the assignment will rely exclusively on RDDs.

### Q1. 
Start by importing Spark and making sure your environment is set up properly for the assignment.

Import the spark context necessary to load a document as an RDD; ignore any error messages

In [2]:
### Write your code here
from pyspark import SparkContext

sc = SparkContext()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/07 01:49:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Q2 

Read in the file `43.txt` as a spark RDD and save it to a variable called `book_43`
 * make sure `book_43` is of type MapPartitionsRDD, i.e.,
   * str(type(book_43)) == "<class 'pyspark.rdd.RDD'>" should return True 

In [3]:
### Write your code here
book_43 = sc.textFile("data/43.txt")

str(type(book_43)) == "<class 'pyspark.rdd.RDD'>"

True

### Q3

How many lines does `book_43` contain?
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [4]:
### Write your code here
book_43_lines = book_43.count()
print(f"Number of lines in Book 43: {book_43_lines}")

Number of lines in Book 43: 2935


### Q4 

Prior to analyzing the words contained in this book, we need to first remove the occurrences of non-alphabetical characters and numbers from the text. You can use the following function, which takes a line as input, removes digits and non-word characters, and splits it into a collection of words. 

```python
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()
```

Use the fucntion above on the variable (test_line) to see what it returns.
```python
test_line = "This is an example of that contains 234 and a dash-containing number"
```

In [5]:
test_line = "This is an example of that contains 234 and a dash-containing number"
def clean_split_line(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.upper().split()

### Write your code here
test_results = clean_split_line(test_line)
print(f"Test Line Results: {test_results}")

Test Line Results: ['THIS', 'IS', 'AN', 'EXAMPLE', 'OF', 'THAT', 'CONTAINS', 'AND', 'A', 'DASH', 'CONTAINING', 'NUMBER']


### Q5

How many words does `book_43` contain? To answer this question, you may find it useful to apply the function in a spark-fashion. 
* You can only use operations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [6]:
### Write your code here
book_43_words = book_43.flatMap(clean_split_line)
book_43_word_count = book_43_words.count()
print(f"Word Count of Book 43: {book_43_word_count}")

Word Count of Book 43: 29116


### Q6

How many of the words in book_43 are unique? Given that words can appear in lower, upper or mixed case (ex. The, THE, the), make sure you convert the words into lower case before counting them.


In [7]:
### Write your code here
# Changed given clean_split_line function to satisfy the lower case counting. 
def clean_split_line2(line):
    a = re.sub('\d+', '', line)
    b = re.sub('[\W]+', ' ', a)
    return b.lower().split()
book_43_uniqe = book_43.flatMap(clean_split_line2).distinct()
book_43_uniqe_count = book_43_uniqe.count()
print(f"Unique Word Count of Book 43: {book_43_uniqe_count}")
book_43_uniqe.take(5)

Unique Word Count of Book 43: 4296


['project', 'gutenberg', 'ebook', 'of', 'strange']

### Q7

* Generate an `RDD` that contains the frequency of each word in `book_43`. Call the variable `book_43_counts`. Each item in the `RDD` should be a tuple with the word as the first element of the tuple and the count as the second item of the tuple. The collection should look like the following:

[('project', 88), ("the", 1807), ... ]

* Such a collection may contain a large number of words and it would be imprudent to transfer all the words onto the same machine to display it. Instead, to explore the content of such a collection, display only the first element in your list. 

* Given the random nature of this operation, the first element element displayed may be different. The first entry for me was:
```
[('project', 88)]
```

* You can only use operations or actions to answer the question. 
* Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
* Code that uses functions such as `some_func(...)` is not allowed


In [8]:
### Write your code here
# Line of Code taken from Week 5 solution
book_43_counts = book_43.flatMap(clean_split_line2).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
book_43_counts.take(1)

[('project', 88)]

### Q8

Sort `book_43_counts` and print the 20 most frequent words in book_43. 
  * Hint: function `sortByKey` sorts a collection of tuples on the first element element of the list. You can easily change the order of the items in each element and use `sortByKey` to sort on the second item of each element in `book_43_counts`
  * You can only use operations or actions to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed

In [9]:
### Write your code here
book_43_sorted = book_43_counts.sortBy(lambda x: x[1], False)
book_43_sorted.take(20)

[('the', 1807),
 ('of', 1068),
 ('and', 1043),
 ('to', 726),
 ('a', 686),
 ('i', 646),
 ('in', 485),
 ('was', 471),
 ('that', 392),
 ('he', 384),
 ('it', 378),
 ('you', 312),
 ('my', 308),
 ('with', 301),
 ('his', 285),
 ('had', 244),
 ('as', 203),
 ('for', 202),
 ('this', 195),
 ('but', 193)]

### Q9

You must have noted that the most frequent words in `book_43_counts` include stop words such as `of`, `the`, `and`, etc.

It would be inefficient to compare documents based on whether or not they contain stop words; those are common to all documents. As such, it's common to remove such stop words. The librarary `sklearn.feature_extraction` provides access to a collection of English stop words, which can be loaded using the following snippet:

```
from sklearn.feature_extraction import stop_words
stop_words.ENGLISH_STOP_WORDS
```

* Explore ENGLISH_STOP_WORDS (it's a frozen set data structure, i.e., a set that you cannot modify) by printing any 10 words from it. 
 * Hint: convert the frozen set to something you can subscript


In [10]:
### Write your code here
from sklearn.feature_extraction import _stop_words
stop_words_list = list(_stop_words.ENGLISH_STOP_WORDS)
print(stop_words_list[:10])

['re', 'out', 'thence', 'yourself', 'neither', 'why', 'once', 'whence', 'wherein', 'ourselves']


### Q10

Filter out the words in `book_43_counts` by removing those that appear in the ENGLISH_STOP_WORDS.
Save the results to a new variable called `book_43_counts_filtered`
  * You can only use operarations or actions on RDDs to answer the question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses function such as `some_func(...)` is not allowed


In [11]:
### Write your code here
book_43_counts_filtered = book_43_counts.filter(lambda x: x[0] not in stop_words_list)
book_43_counts_filtered.take(5)

[('project', 88),
 ('gutenberg', 98),
 ('ebook', 13),
 ('strange', 24),
 ('mr', 128)]

### Q11

* How many words are left in `book_43_counts_filtered` after removing the stop words

In [12]:
### Write your code here
book_43_filtered_counts = book_43_counts_filtered.count()
print(f"Book 43 Filtered Count of Words: {book_43_filtered_counts}")

Book 43 Filtered Count of Words: 4034


### Q12 

* Create a function called *process_RDD* that combines the relevant steps you proposed above to make it convenient to apply them to the remaining four books. Your function should accept an input text file path and:
 * Reads in the file as a textRDD
 * Cleans and splits the line using `clean_split_line`
 * Filters out the stop words
 * Returns a word count RDD where each item is a tuple of words and its count.
 


In [13]:
### Write your code here
def process_RDD(text_file_path):
    ### Takes the text file path and reads it as an RDD. Question 2
    new_book = sc.textFile(text_file_path)
    ### Takes the RDD and splits it into a tuple of words and their count. Question 7
    book_line_split = new_book.flatMap(clean_split_line2).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
    ### Takes the Tuples of words and counts and filters out the stop words. Question 10
    book_filtered_word_count = book_line_split.filter(lambda x: x[0] not in stop_words_list)
    return book_filtered_word_count
### Example used for function.
example = process_RDD("data/84.txt")
example.take(5)

[('project', 88),
 ('gutenberg', 98),
 ('ebook', 13),
 ('mary', 3),
 ('shelley', 3)]

### Q13 

Apply the function `process_RDD` to `book_84`, `book_398` and `book_3296` and save the results to variables `book_84_counts_filtered`, `book_398_counts_filtered` and `book_3296_counts_filtered` respectively. How many distinct words does each book contain after filtering the stop words.


In [14]:
### Write your code here
book_84_counts_filtered = process_RDD("data/84.txt")
book_398_counts_filtered = process_RDD("data/398.txt")
book_3296_counts_filtered = process_RDD("data/3296.txt")

book_84_distinct = book_84_counts_filtered.count()
book_398_distinct = book_398_counts_filtered.count()
book_3296_distinct = book_3296_counts_filtered.count()
print(f"Distinct words for book 84 ={book_84_distinct}\nDistinct words for book 398 ={book_398_distinct}\nDistinct words for book 3296 ={book_3296_distinct}\n")

Distinct words for book 84 =7016
Distinct words for book 398 =2421
Distinct words for book 3296 =7293



### Q14 

We discussed how to evaluate similarity between two texts using the number of words they share. We hypothesized that books that are similar should have more words in common than books that are dissimilar. If that holds, `book_398` and `book_3296`, which both pertain to religion, will have more words in common than, say, `book_84` and `book_398`. Test this hypothesis by writing code that compares and prints the number of words shared between `book_398` and `book_3296` and then between `book_84` and `book_398`.


In [15]:
### Write your code here
def book_comparison(book_one, book_two):
    word_comparison_count = book_one.join(book_two).count()
    return word_comparison_count

comparison_398_3296 = book_comparison(book_398_counts_filtered, book_3296_counts_filtered)
comparison_84_398 = book_comparison(book_84_counts_filtered, book_398_counts_filtered)
print(f"Number of similar words between book 398 and book 3296: {comparison_398_3296}\nNumber of simialr words between book 84 and book 398: {comparison_84_398}")

comparison_84_3296 = book_comparison(book_84_counts_filtered, book_3296_counts_filtered)
print(f"Number of fimilar words between book 84 and book 3296: {comparison_84_3296}")

Number of similar words between book 398 and book 3296: 1790
Number of simialr words between book 84 and book 398: 1691
Number of fimilar words between book 84 and book 3296: 3608


### Q15

* Based on the above, do you think counting the number of shared words is a good idea as a distance metric for evaluating topic similarity? Justify your answer?
* Hint: What do *book_84* and *book_3296* have in common? 

#### Write your answer here

No not necessarily, because the numeric value of the comparisons may not actually prove the evaluation of topic similarity. Rather the ratio of comparisons over the number of words from both books may be a better evaluation metric instead since it accounts for the length of both books possibly being a factor. With that being said it still isn't very accurate either, as discussed in class today(Nov 1.) collecting and seperating the generalized words from the list of shared words to aquire the unique shared words may be a better evaluation process to determine topic similarity.

## Part II 

Another approach to estimating similarity consists of computing the Euclidean distance across a set of words. For example, suppose we have 3 documents A, B and C with the following counts for the words `evolution`, `DNA`, `biology` and `finance`. 

```python 
A = [4, 9, 6, 8]
B = [3, 7, 7, 10]
C = [15, 10, 1, 1]
```
Although all documents contain exactly the four words, the number of times these words appear in each book may be indicative of thier topic. For example, documents `A` and `B` are more likely to be business related since they contain the word `finance` more frequently (8 and 10 times respectively). Document `C` may be a technical document since it focuses on more technical words (`evolution` and `DNA`) and less on the words `finance`.

The Euclidean distance, which can be computed using the `scikit` snippet below, is more indicative of topic-relatedness between the two documents.

```python
from scipy.spatial.distance import euclidean 
print(f"The Euclidean distance between A and B is: {euclidean(A, B)}")

print(f"The Euclidean distance between A and C is: {euclidean(A, C)}")

print(f"The Euclidean distance between B and C is: {euclidean(B, C)}")
```


### Q16

To calculate the Euclidean distance, we must first identify the set of words by which we will compare the documents. Here, we will explore the words that are common to all 4 documents. We will store the data in a matrix called `counts_matrix`.

Start by finding the words that are common to all four documents after stop-word filtering and store the counts for each word in a column of `counts_matrix`. 

To take the previous example, you can generate an emtpy matrix with 3 lines (documents `A`, `B` and `C`) and 4 columns (words `evolution`, `DNA`, `biology` and `finance`) using the following code.

```python
import numpy as np
counts_matrix = np.zeros([3,4])
```

After generting the counts, you can fill the counts for a document, say `A`, using the following code:

```python
counts_matrix[0, :] = [4, 9, 6, 8] 
```
* Other than for building `counts_matrix` you should exclusively use operations or actions on the `RDD` to answer this question. 
  * Code that uses methods such as `some_rdd.X().Y().Z()...` is allowed
  * Code that uses functions such as `some_func(...)` is not allowed


In [16]:
### Write your code here
import numpy as np

book_43_counts_filtered = process_RDD("data/43.txt")
book_84_counts_filtered = process_RDD("data/84.txt")
book_398_counts_filtered = process_RDD("data/398.txt")
book_3296_counts_filtered = process_RDD("data/3296.txt")

four_book_word_similarity = book_43_counts_filtered.join(book_84_counts_filtered).join(book_398_counts_filtered).join(book_3296_counts_filtered)
matrix_lines = four_book_word_similarity.count()
counts_matrix = np.zeros([4,matrix_lines])
counts_matrix[0, :] = four_book_word_similarity.map(lambda x: x[1][0][0][0]).collect()
counts_matrix[1, :] = four_book_word_similarity.map(lambda x: x[1][0][0][1]).collect()
counts_matrix[2, :] = four_book_word_similarity.map(lambda x: x[1][0][1]).collect()
counts_matrix[3, :] = four_book_word_similarity.map(lambda x: x[1][1]).collect()

counts_matrix

array([[88., 98., 13., ...,  1.,  1.,  1.],
       [88., 98., 13., ...,  2.,  2.,  1.],
       [87., 93.,  9., ...,  1.,  1.,  1.],
       [88., 97., 10., ...,  1.,  1.,  1.]])

### Q17

Compute the Euclidean distance between `book_398` and `book_3296`, which both talk about religion and `book_84` and `book_398`. What do you conclude about using the Euclidean distance for evaluating topic relatedness across documents?


In [17]:
### Write your code here
from scipy.spatial.distance import euclidean
print(f"Euclidean Distance between book 398 and book 3296: {format(euclidean(counts_matrix[2], counts_matrix[3]), '.2f')}")
print(f"Euclidean Distance between book 84 and book 398: {format(euclidean(counts_matrix[1], counts_matrix[2]), '.2f')}")

Euclidean Distance between book 398 and book 3296: 1156.66
Euclidean Distance between book 84 and book 398: 751.67


#### Write your justification here

The euclidean distance wasn't a useful metric to measure topic relatedness across documents since it proved that the two books related to religion still aren't as similar as one with another topic. This can be identified by the larger distance between the two religious books over comparison with the third book.

### Q18

Bonus question (5 points): Can you think of a few things we could do to improve similarity between documents that pertain to the same topic. Justify your answer without giving code

#### Write your answer here

As discussed in question 15 and 17, simple comaprisons based on word quantities have been less than successful and a focus on unique words may be an initial solution to the issue of topic similarity. I would also argue that using a ratio rather than a count may also assist. For example, when comparing three or more books to determine if two may share a topic, all the shared words between the books can be grouped to form a general list of distinct words. This list can then be used to determine the unique words any two books share in order to form a ratio to base a comparison on to ensure comparisons between books with more words aren't unequally compared.

## Part III

In this part we will build some basic analytics for a dataset consisting of flight arrival and departure details for all commercial flights within the USA in one month. While this dataset can be managed using Pandas (<1M records), scaling to a yearly or longer timeframe will greatly benefit from using a distributed computing framework such as `Spark`.

Here, you should use exclusively `SparkDatFrames. 

We want to analyze this dataset to better schedule trips.  For example:
 * Avoid airlines carriers that are most often associated with delays.
 * Avoid departure days where delays are most frequent.
 * Avoid airports which are associated with delays or long taxxying time.
* etc.
 

The information about the fields contained in the data file can be found [here](https://dataverse.harvard.edu/dataset.xhtml;jsessionid=0414e25969eccd0e88ae4d64fa0b?persistentId=doi%3A10.7910%2FDVN%2FHG7NV7&version=&q=&fileTypeGroupFacet=&fileTag=%221.+Documentation%22&fileSortField=date&fileSortOrder=desc)


### Q19

Load the file `flight_info.csv` into a spark `DataFrame` called `fight_info`.

  * Note that you will need to create a sparkSession prior to loading the data
  
* How many entries does the file contain?



In [18]:
### Write your code here
from pyspark.sql import SparkSession

# sparkSession created using Spark Apache documentation. 
spark = SparkSession.builder.getOrCreate()

# Load the csv using text file
fight_info = spark.read.csv("data/flight_info.csv", header=True)
# Use .count() to count the number of entries in the file.
fight_info.show()

+---+---------+-------------+---------+------+----+----------+-------+-------+---------+--------+------+----------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|_c0|DayOfWeek|UniqueCarrier|FlightNum|Origin|Dest|CRSDepTime|DepTime|TaxiOut|WheelsOff|WheelsOn|TaxiIn|CRSArrTime|ArrTime|Cancelled|CancellationCode|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+---------+-------------+---------+------+----+----------+-------+-------+---------+--------+------+----------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|        2|           AA|      494|   CLT| PHX|      1619| 1616.0|   17.0|   1633.0|  1837.0|   5.0|      1856| 1842.0|      0.0|            null|  1773.0|        null|        null|    null|         null|             null|
|  1|        3|           AA|      494|   CLT| PHX|      1619| 1614.0|   13.0|   1627.0|  18

21/11/07 01:50:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
 Schema: _c0, DayOfWeek, UniqueCarrier, FlightNum, Origin, Dest, CRSDepTime, DepTime, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, Cancelled, CancellationCode, Distance, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay
Expected: _c0 but found: 
CSV file: file:///home/jovyan/work/assignment-2-nicholasmp/data/flight_info.csv


### Q20

Use `pySpark-SQL` or `pandas-like syntax to compute the airlines represented in this dataset
The airline information is stored in a field called UniqueCarrier
* UniqueCarrier: Represents the unique carrier code (ex.AA = American Airlines) 


In [19]:
### Write your code here
unique_carrier = fight_info.select("UniqueCarrier").show(10)

+-------------+
|UniqueCarrier|
+-------------+
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
|           AA|
+-------------+
only showing top 10 rows



### Q21

The data file contains various other fields, two of which are useful for answering the next question.

* CRSDepTime: Represents the scheduled departure time
* DepTime: Represents the actual departure time

Compute the number of flights delayed per each carried code represented in this dataset. Sort the data by decreasing order of delays.
  * A delay is observed when `DepTime` > `CRSDepTime`


In [22]:
### Write your code here
# select the three fields in question and compare them using .where() the obsereved delay occurs.
delayed_flights = fight_info.select("UniqueCarrier", "DepTime", "CRSDepTime").where("DepTime > CRSDepTime")
# group the observed delays by the Carrier Code and use the built in count aggregate function.
# sort in descending order by the count of delays and collect the data.
grouped_delays = delayed_flights.groupBy("UniqueCarrier").agg({"*": "count"}).sort("count(1)", ascending=False).collect()
# print this output nicely.
print(f"Total delays: {delayed_flights.count()}")
for row in grouped_delays:
    print("Carrier Code: " + row['UniqueCarrier'] + ", Delayed Flight Count: " + str(row['count(1)']))

Total delays: 187792
Carrier Code: WN, Delayed Flight Count: 54557
Carrier Code: DL, Delayed Flight Count: 28962
Carrier Code: AA, Delayed Flight Count: 26291
Carrier Code: UA, Delayed Flight Count: 19594
Carrier Code: OO, Delayed Flight Count: 17924
Carrier Code: EV, Delayed Flight Count: 12340
Carrier Code: B6, Delayed Flight Count: 10406
Carrier Code: AS, Delayed Flight Count: 4966
Carrier Code: NK, Delayed Flight Count: 4435
Carrier Code: F9, Delayed Flight Count: 3181
Carrier Code: VX, Delayed Flight Count: 2871
Carrier Code: HA, Delayed Flight Count: 2265


### Q22

 Use the file `airlines.csv` to find the the complete name of the airline. Here, you are required to load the file as a pyspark DataFrame; call it `airlines_info`, and repeat the query above while including the `flights.csv `file in your query ( requires doing a `join`) so that you can also display the full name of the carrier (second column). 

The result will look (approximately) like:

```
[Row(UniqueCarrier='WN', first(_c1)='Southwest Airlines', count=SOME_count),
 Row(UniqueCarrier='DL', first(_c1)='Delta Air Lines', count=SOME_count),
 Row(UniqueCarrier='AA', first(_c1)='American Airlines', count=SOME_count),
 ...
 ]
```

The carrier code in the `airlines.csv` file is provided in the 4th (1-based) column

Note that the file `airlines.csv` does not have column header. Hence, you need to print one line of your dataset to see what names Spark gave to the columns. Use the name provided by Spark in your query.

In [39]:
### Write your code here
airlines_info = spark.read.csv("data/airlines.csv", header=False)
delayed_flights_two = fight_info.join(airlines_info, fight_info.UniqueCarrier == airlines_info._c3).select("UniqueCarrier", "_c1", "DepTime", "CRSDepTime").where("DepTime > CRSDepTime")
full_grouped_delays = delayed_flights_two.groupBy(["UniqueCarrier", "_c1"]).agg({"*": "count"}).sort("count(1)", ascending=False).collect()
# full_grouped_delays
for row in full_grouped_delays:
    print("Carrier Code: " + row['UniqueCarrier'] + "_c1='" + row['_c1'] + "', Delayed Flight Count: " + str(row['count(1)']))
    

Carrier Code: WN_c1='Southwest Airlines', Delayed Flight Count: 54557
Carrier Code: DL_c1='Delta Air Lines', Delayed Flight Count: 28962
Carrier Code: AA_c1='American Airlines', Delayed Flight Count: 26291
Carrier Code: UA_c1='United Airlines', Delayed Flight Count: 19594
Carrier Code: OO_c1='SkyWest', Delayed Flight Count: 17924
Carrier Code: EV_c1='Atlantic Southeast Airlines', Delayed Flight Count: 12340
Carrier Code: B6_c1='JetBlue Airways', Delayed Flight Count: 10406
Carrier Code: AS_c1='Alaska Airlines', Delayed Flight Count: 4966
Carrier Code: NK_c1='Spirit Airlines', Delayed Flight Count: 4435
Carrier Code: F9_c1='Frontier Airlines', Delayed Flight Count: 3181
Carrier Code: VX_c1='Virgin America', Delayed Flight Count: 2871
Carrier Code: HA_c1='Hawaiian Airlines', Delayed Flight Count: 2265


### Q23

Compute the number of delays per company per day. The day is encoded as an integer in the column `DayOfWeek` in `fight_info`. You can display the day as an integer or map it into a string name of the week.
Sort the data by airline code (UniqueCarrier) and by increasing values of DayOfWeek


You results should look like the following



In [ ]:
### Write your code here

### Q24  

Counting the number of delayed flights per airline is misleading, as airlines with more flights are more likley to have delays than companies with substantially fiewer flights. 

Repeat the same query above but, for each carrier, normalize the counts of delays by the total number of flights for that carrier. 


In [ ]:
### Write your code here

### Q25 

Time the query above. How long did it take to run. 
  * Make sure you run the code a few times and compute the average run time.
  * The above should be easy to implement if you use the correct Jupyter Notebook `magic` function
  

In [ ]:
### Write your code here

### Q26 

Use one of the techniques covered in class to accelerate this query. Time your query to see by how much the run time was improved

In [ ]:
### Write your code here

### Q27 

Is the departure delay (i.e., DepTime - CRSDepTime) predictive of the arrival delay (ArrTime > CRSArrTime)?
Use an approach of your choice (e.g. `skelearn` which we covered in class or `Spark`) to model as a linear regression the arrival delay as a function of the departure delay. 



In [ ]:
### Write your code here